## Working

In [104]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
drive.mount('/gdrive')

df=pd.read_csv('/gdrive/My Drive/FL/Trading/v2.csv')

df['fc'] = pd.to_datetime(df['fc'], errors='coerce')
month = df['fc'].dt.strftime('%Y%m')
month = pd.to_numeric(month)
if 'month' not in df:
  df.insert (1, "month", month)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [106]:
SL = 0.06
TG = 0.15
dias_indeterminacion = 90
df['target_alza'] = 99
df['target_baja'] = 99

i = 1
while i <= dias_indeterminacion:
  var_y_low = df.low.shift(-i)/df.y-1 # Variación del mínimo de cada día contra el precio de compra
  var_y_high = df.high.shift(-i)/df.y-1 # Variación del máximo de cada día contra el precio de compra
  target_alza = np.where(var_y_low < -SL, 0, 99)
  target_alza = np.where(var_y_high > TG, 1, target_alza)
  df['target_alza'] = np.where(df['target_alza'] == 99 , target_alza , df['target_alza'])
  target_baja = np.where(var_y_low < -TG, 1, 99)
  target_baja = np.where(var_y_high > SL, 0, target_baja)
  df['target_baja'] = np.where(df['target_baja'] == 99 , target_baja , df['target_baja'])
  i = i + 1

### XGBoost

In [108]:
base = df.sort_values(by=['fc'])
base = base[(base.target_alza + base.target_baja) < 90] # Elimina indeterminados
un_decimo = int(len(base)/10)
tres_decimos = int(len(base)/10)*3
siete_decimos = int(len(base)/10)*7
base_params = base.iloc[un_decimo:tres_decimos]
base_train = base.iloc[dos_decimos:siete_decimos]
base_test = base.iloc[siete_decimos:(len(base)-180)]

In [109]:
# Parametrizo
X=base_params.iloc[:, 10:].drop(["target_alza", "target_baja"], 1)
targets=base_params.target_alza
pred_train, pred_test, tar_train, tar_test = train_test_split(X, targets, test_size=.4)

eval_set = [(pred_train, tar_train),(pred_test, tar_test)]
bst=XGBClassifier(objective= 'binary:logistic',seed=5)
params = {  
    "n_estimators": st.randint(15, 100),
    "max_depth": st.randint(3, 4),
    "learning_rate": st.uniform(0.05, 0.3),
    "colsample_bytree": st.beta(10, 1) ,
    "subsample": st.beta(10, 1)  ,
    "gamma": st.uniform(0, 10),
    'reg_alpha': st.expon(0, 50),
    "min_child_weight": st.expon(0, 50)
          }
gs = RandomizedSearchCV(bst, params,  n_iter=10, n_jobs=1,cv=4 )  
bst=gs.fit(pred_train, tar_train,eval_set=eval_set, eval_metric=["auc"],early_stopping_rounds=10) 

# Entreno
X=base_train.iloc[:, 10:].drop(["target_alza", "target_baja"], 1)
targets=base_train.target_alza
pred_train, pred_test, tar_train, tar_test = train_test_split(X, targets, test_size=.4)
eval_set = [(pred_train, tar_train),(pred_test, tar_test)]
classif = XGBClassifier(**bst.best_params_)
model_alza=classif.fit(pred_train,tar_train,eval_set=eval_set, eval_metric=["auc"],early_stopping_rounds=10)

# Parametrizo
X=base_params.iloc[:, 10:].drop(["target_alza", "target_baja"], 1)
targets=base_params.target_baja
pred_train, pred_test, tar_train, tar_test = train_test_split(X, targets, test_size=.4)

eval_set = [(pred_train, tar_train),(pred_test, tar_test)]
bst=XGBClassifier(objective= 'binary:logistic',seed=5)
params = {  
    "n_estimators": st.randint(15, 100),
    "max_depth": st.randint(3, 4),
    "learning_rate": st.uniform(0.05, 0.3),
    "colsample_bytree": st.beta(10, 1) ,
    "subsample": st.beta(10, 1)  ,
    "gamma": st.uniform(0, 10),
    'reg_alpha': st.expon(0, 50),
    "min_child_weight": st.expon(0, 50)
          }
gs = RandomizedSearchCV(bst, params,  n_iter=10, n_jobs=1,cv=4 )  
bst=gs.fit(pred_train, tar_train,eval_set=eval_set, eval_metric=["auc"],early_stopping_rounds=10) 

# Entreno
X=base_train.iloc[:, 10:].drop(["target_alza", "target_baja"], 1)
targets=base_train.target_baja
pred_train, pred_test, tar_train, tar_test = train_test_split(X, targets, test_size=.4)
eval_set = [(pred_train, tar_train),(pred_test, tar_test)]
classif = XGBClassifier(**bst.best_params_)
model_baja=classif.fit(pred_train,tar_train,eval_set=eval_set, eval_metric=["auc"],early_stopping_rounds=10)


[0]	validation_0-auc:0.625506	validation_1-auc:0.621665
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.68389	validation_1-auc:0.665858
[2]	validation_0-auc:0.719389	validation_1-auc:0.695189
[3]	validation_0-auc:0.728052	validation_1-auc:0.698356
[4]	validation_0-auc:0.736517	validation_1-auc:0.707069
[5]	validation_0-auc:0.746195	validation_1-auc:0.713769
[6]	validation_0-auc:0.749978	validation_1-auc:0.72205
[7]	validation_0-auc:0.754812	validation_1-auc:0.727625
[8]	validation_0-auc:0.766403	validation_1-auc:0.738554
[9]	validation_0-auc:0.769731	validation_1-auc:0.740617
[10]	validation_0-auc:0.771978	validation_1-auc:0.743231
[11]	validation_0-auc:0.774732	validation_1-auc:0.746642
[12]	validation_0-auc:0.778953	validation_1-auc:0.751009
[13]	validation_0-auc:0.780387	validation_1-auc:0.751634
[14]	validation_0-auc:0.783726	validation_1-auc:0.755246


In [113]:
# Test
dias = 30
X=base_test.iloc[:, 10:].drop(["target_alza", "target_baja"], 1)
preds_alza = model_alza.predict_proba(X)
preds_baja = model_baja.predict_proba(X)
oos = base_test.iloc[:, 2:5]
oos['pred_alza'] = preds_alza[:,1]
oos['pred_baja'] = preds_baja[:,1]
oos = oos.sort_values(["ticker", "fc"], ascending = (False, True))
oos['result'] = (oos.y.shift(-dias)/oos.y) - 1
oos['result'] = np.where(oos.ticker != oos.ticker.shift(-dias), np.nan, oos.result)
oos['result_pn'] = np.where(oos.result > 0, "Alza", "Baja")
oos['result_pn'] = np.where(oos.ticker != oos.ticker.shift(-dias), np.nan, oos.result_pn)
oos = oos.sort_values(["fc"], ascending = (True))
oos['bucket_alza'] = pd.qcut(oos['pred_alza'].values, 10, duplicates='drop')
oos['bucket_baja'] = pd.qcut(oos['pred_baja'].values, 10, duplicates='drop')


In [114]:
print("Cantidad:")
oos.groupby(['bucket_alza', 'bucket_baja']).agg({'result': ['count']}).unstack()

Cantidad:


result  ...               
                                                      count  ...               
bucket_baja                   (0.021599999999999998, 0.114]  ... (0.448, 0.753]
bucket_alza                                                  ...               
(0.030899999999999997, 0.165]                            17  ...            635
(0.165, 0.205]                                           63  ...            479
(0.205, 0.242]                                           81  ...            364
(0.242, 0.28]                                           112  ...            272
(0.28, 0.321]                                           148  ...            166
(0.321, 0.367]                                          202  ...            145
(0.367, 0.42]                                           244  ...             93
(0.42, 0.483]                                           310  ...             97
(0.483, 0.584]                                          313  ...             75
(0.584, 0.997]                                          757  ...             82

[10 rows x 10 columns]

In [115]:
print("Promedios:")
oos.groupby(['bucket_alza', 'bucket_baja']).agg({'result': ['median']}).unstack()


Promedios:


result  ...               
                                                     median  ...               
bucket_baja                   (0.021599999999999998, 0.114]  ... (0.448, 0.753]
bucket_alza                                                  ...               
(0.030899999999999997, 0.165]                      0.100000  ...      -0.012953
(0.165, 0.205]                                     0.039370  ...       0.024134
(0.205, 0.242]                                     0.003472  ...       0.032406
(0.242, 0.28]                                      0.033205  ...       0.017551
(0.28, 0.321]                                      0.026214  ...       0.001258
(0.321, 0.367]                                     0.042603  ...      -0.044828
(0.367, 0.42]                                      0.015298  ...      -0.034302
(0.42, 0.483]                                      0.028099  ...       0.038627
(0.483, 0.584]                                     0.065020  ...       0.048544
(0.584, 0.997]                                     0.056040  ...       0.000000

[10 rows x 10 columns]

In [122]:
pd.set_option('display.max_rows', 1500)
oos[(oos['pred_alza']>0.5)&(oos['pred_baja']<0.11)]

,fc,ticker,y,pred_alza,pred_baja,result,result_pn,bucket_alza,bucket_baja
52090,2016-12-29,CARC.BA,1.740000,0.838389,0.034788,0.293103,Alza,"(0.584, 0.997]","(0.021599999999999998, 0.114]"
52091,2017-01-02,CARC.BA,1.820000,0.526913,0.037930,0.197802,Alza,"(0.483, 0.584]","(0.021599999999999998, 0.114]"
19539,2017-01-04,EDN.BA,24.700001,0.579337,0.100107,0.078947,Alza,"(0.483, 0.584]","(0.021599999999999998, 0.114]"
19540,2017-01-05,EDN.BA,24.700001,0.570201,0.100107,0.054656,Alza,"(0.483, 0.584]","(0.021599999999999998, 0.114]"
19542,2017-01-09,EDN.BA,24.500000,0.523201,0.099863,0.053061,Alza,"(0.483, 0.584]","(0.021599999999999998, 0.114]"
66897,2017-01-10,LEDE.BA,17.150000,0.567962,0.104995,0.000000,Baja,"(0.483, 0.584]","(0.021599999999999998, 0.114]"
66898,2017-01-11,LEDE.BA,16.500000,0.621820,0.105441,0.027273,Alza,"(0.584, 0.997]","(0.021599999999999998, 0.114]"
52101,2017-01-16,CARC.BA,2.180000,0.535148,0.100880,-0.022936,Baja,"(0.483, 0.584]","(0.021599999999999998, 0.114]"
52102,2017-01-17,CARC.BA,2.200000,0.563173,0.094422,-0.022727,Baja,"(0.483, 0.584]","(0.021599999999999998, 0.114]"
52103,2017-01-18,CARC.BA,2.150000,0.735754,0.093077,-0.037209,Baja,"(0.584, 0.997]","(0.021599999999999998, 0.114]"


Compro cuando la pred_alza supere 0.5 y la pred_baja sea menor a 0.11

In [124]:
import pickle
pickle.dump(model_alza, open("model_alza.dat", "wb"))
pickle.dump(model_baja, open("model_baja.dat", "wb"))